In [ ]:
#Standaard imports
import pickle

import numpy as np
import pandas as pd
from pandas_summary import DataFrameSummary
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm
from feature_engine.encoding import OneHotEncoder
from sklearn import set_config
from sklearn.base import clone
from sklearn.ensemble import BaggingRegressor
from sklearn.preprocessing import MinMaxScaler
from feature_engine.encoding import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from skmultilearn.problem_transform import LabelPowerset
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier

In [ ]:
dft= pd.read_csv("testear.csv")

In [ ]:
dft= dft.drop('Unnamed: 0', axis=1) ##Voorbereiden van test data

In [ ]:
dft= dft.dropna(subset=['genres_x'])

In [ ]:
dft['endYear'] = dft['endYear'].replace(0, 2023)
dft['Years'] = dft['endYear'] - dft['startYear']
dft = dft.drop(columns=['startYear', 'endYear'])

In [ ]:
discrete_columns = []
numerical_columns = [c for c in df.columns if df[c].dtype != 'O' and c not in discrete_columns]
categorical_columns = [c for c in df.columns if df[c].dtype == 'O']

In [ ]:
discrete_columns = []
numerical_columns = [c for c in df.columns if df[c].dtype != 'O' and c not in discrete_columns]
categorical_columns = [c for c in df.columns if df[c].dtype == 'O']

In [ ]:
dft_num = dft[numerical_columns]

In [ ]:
dft_cat = dft[categorical_columns]

In [ ]:
##Data pre-processing
def PreProc(df_num):
    df_num.loc[:, 'episodeNumber'] = df_num.loc[:, 'episodeNumber'] .fillna(1)
    df_num.loc[df_num.loc[:, 'episodeNumber']  == 0, 'episodeNumber'] = 1 #Every  episode that is zero or NaN is 1 --> atleast one chapter for a program
    df_num.loc[:, 'seasonNumber'] = df_num.loc[:, 'seasonNumber'].fillna(0) #Every Nan season number gets the value 0
    return df_num

In [ ]:
dft_num = PreProc(dft_num)

In [ ]:
col_idx = ['directors', 'writers']

In [ ]:
for col in col_idx:
    dft_cat[col] = dft_cat[col].str.lstrip('nm')
    dft_cat[col] = dft_cat[col].str.split(',').str[0]

In [ ]:
dft_cat = pd.get_dummies(dft_cat, columns=['titleType'], prefix=['titleType'])

In [ ]:
dft_cat['genres_x'] = dft_cat['genres_x'].str.split(',')
dft_cat = pd.concat([dft_cat, dft_cat['genres_x'].apply(lambda x: pd.Series([1] * len(x), index=x)).fillna(0)], axis=1)

In [ ]:
dft_cat.drop('genres_x', axis=1, inplace=True)

In [ ]:
num_6_col = ['numVotes','isAdult', 'Years', 'runtimeMinutes','seasonNumber','episodeNumber']

In [ ]:
dft_num_6 = dft_num[num_6_col]

In [ ]:
columns_to_exclude = ['language', 'adult', 'genres_y', 'original_language', 'production_companies', 'production_countries','attributes', 'status','tagline','video']
dft_cat_fin = dft_cat.drop(columns=columns_to_exclude)

In [ ]:
final_test = pd.concat([dft_num_6,dft_cat_fin], axis=1)

In [ ]:
column_list = ['numVotes', 'isAdult', 'Years', 'runtimeMinutes', 'seasonNumber',
       'episodeNumber', 'directors', 'writers', 'titleType_movie',
       'titleType_short', 'titleType_tvEpisode', 'titleType_tvMiniSeries',
       'titleType_tvMovie', 'titleType_tvSeries', 'titleType_tvShort',
       'titleType_tvSpecial', 'titleType_video', 'titleType_videoGame',
       'Comedy', 'Musical', 'Action', 'Adventure', 'Animation', 'Documentary',
       'History', 'War', 'Drama', 'Mystery', 'Romance', 'Short', 'Music',
       'Talk-Show', 'Crime', 'Horror', 'Biography', 'Western', 'Sci-Fi',
       'Sport', 'Game-Show', 'News', 'Fantasy', 'Family', 'Reality-TV', '0',
       'Adult', 'Thriller', 'Film-Noir']

In [ ]:
final_test = final_test.reindex(columns=column_list)

In [ ]:
with open('saved_model.pkl', 'rb') as file:
    model = pickle.load(file)

In [ ]:
predictions = model.predict(final_test)

In [ ]:
submission = pd.DataFrame(predictions, columns=["averageRating"])

In [ ]:
submission.index.name = "Id"

In [ ]:
submission.to_csv("submission.csv")